In [17]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [18]:
# training_data = datasets.MNIST(
#     root="data",
#     train=True,
#     download=True,
#     transform=ToTensor(),
# )
# test_data = datasets.MNIST(
#     root="data",
#     train=False,
#     download=True,
#     transform=ToTensor(),
# )
training_data = datasets.EMNIST(
    root="data",
    split="byclass",
    train=True,
    download=True,
    transform=ToTensor(),
)
test_data = datasets.EMNIST(
    root="data",
    split="byclass",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [19]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break
    
n_classes = len(training_data.targets.unique())

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [20]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, stride=1, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=32),
            nn.Dropout(p=0.4),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=5, stride=1, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(num_features=64),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.4),
            nn.Linear(in_features=64*20*20, out_features=128),
            nn.ReLU(),
            nn.BatchNorm1d(num_features=128),
            nn.Dropout(p=0.4),
            nn.Linear(in_features=128, out_features=n_classes),
        )

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

model = CNN().to(device)
print(model)

Using cuda device
CNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (7): ReLU()
    (8): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Dropout(p=0.4, inplace=False)
    (10): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (11): ReLU()
    (12): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (14): ReLU()
    (15): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (16): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=same)

In [21]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

In [22]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [23]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [24]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 4.385159  [   64/697932]
loss: 1.321488  [ 6464/697932]
loss: 0.773793  [12864/697932]
loss: 0.764543  [19264/697932]
loss: 0.458771  [25664/697932]
loss: 0.503438  [32064/697932]
loss: 0.763708  [38464/697932]
loss: 0.703707  [44864/697932]
loss: 0.431163  [51264/697932]
loss: 0.506679  [57664/697932]
loss: 0.299994  [64064/697932]
loss: 0.369593  [70464/697932]
loss: 0.667321  [76864/697932]
loss: 0.492528  [83264/697932]
loss: 0.730648  [89664/697932]
loss: 0.624056  [96064/697932]
loss: 0.499478  [102464/697932]
loss: 0.705186  [108864/697932]
loss: 0.556412  [115264/697932]
loss: 0.531247  [121664/697932]
loss: 0.494656  [128064/697932]
loss: 0.439551  [134464/697932]
loss: 0.567941  [140864/697932]
loss: 0.401271  [147264/697932]
loss: 0.407912  [153664/697932]
loss: 0.467192  [160064/697932]
loss: 0.332622  [166464/697932]
loss: 0.568615  [172864/697932]
loss: 0.371175  [179264/697932]
loss: 0.429263  [185664/697932]
loss: 0.451386  

loss: 0.342768  [198464/697932]
loss: 0.356100  [204864/697932]
loss: 0.322858  [211264/697932]
loss: 0.346377  [217664/697932]
loss: 0.451216  [224064/697932]
loss: 0.360533  [230464/697932]
loss: 0.444922  [236864/697932]
loss: 0.166199  [243264/697932]
loss: 0.308297  [249664/697932]
loss: 0.438564  [256064/697932]
loss: 0.175498  [262464/697932]
loss: 0.536731  [268864/697932]
loss: 0.432325  [275264/697932]
loss: 0.562841  [281664/697932]
loss: 0.527924  [288064/697932]
loss: 0.366313  [294464/697932]
loss: 0.373656  [300864/697932]
loss: 0.309666  [307264/697932]
loss: 0.264120  [313664/697932]
loss: 0.345588  [320064/697932]
loss: 0.323735  [326464/697932]
loss: 0.508933  [332864/697932]
loss: 0.397446  [339264/697932]
loss: 0.507699  [345664/697932]
loss: 0.353919  [352064/697932]
loss: 0.346440  [358464/697932]
loss: 0.231578  [364864/697932]
loss: 0.388840  [371264/697932]
loss: 0.287885  [377664/697932]
loss: 0.443243  [384064/697932]
loss: 0.184224  [390464/697932]
loss: 0.

loss: 0.399342  [403264/697932]
loss: 0.331273  [409664/697932]
loss: 0.362899  [416064/697932]
loss: 0.352771  [422464/697932]
loss: 0.392485  [428864/697932]
loss: 0.397501  [435264/697932]
loss: 0.335496  [441664/697932]
loss: 0.318993  [448064/697932]
loss: 0.245892  [454464/697932]
loss: 0.357616  [460864/697932]
loss: 0.347752  [467264/697932]
loss: 0.345797  [473664/697932]
loss: 0.269702  [480064/697932]
loss: 0.267302  [486464/697932]
loss: 0.197686  [492864/697932]
loss: 0.385217  [499264/697932]
loss: 0.258176  [505664/697932]
loss: 0.441872  [512064/697932]
loss: 0.351835  [518464/697932]
loss: 0.288802  [524864/697932]
loss: 0.506034  [531264/697932]
loss: 0.231193  [537664/697932]
loss: 0.210780  [544064/697932]
loss: 0.380508  [550464/697932]
loss: 0.690685  [556864/697932]
loss: 0.278445  [563264/697932]
loss: 0.410609  [569664/697932]
loss: 0.366827  [576064/697932]
loss: 0.318611  [582464/697932]
loss: 0.356046  [588864/697932]
loss: 0.386134  [595264/697932]
loss: 0.

In [25]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [26]:
model = CNN()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [27]:
classes = [i for i in range(len(training_data.targets))]

model.eval()
x, y = test_data[0]
with torch.no_grad():
    x = x.unsqueeze(0)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "1", Actual: "18"
